In [1]:
s = 100
ms = 3
xx = randn(Float32,s,s)
mm = randn(Float32,ms,ms)
gg = randn(Float32,s,s*ms)
nx = zeros(Float32,size(mm,2)*size(xx,2),size(xx,1)-size(mm,1)+1)
M,N = size(xx)
n = 1
m = size(mm,2)
B = Array{eltype(Matrix{Float32})}(undef, m*n,(M-m+1)*(N-n+1))

include("../MyReverseDiff.jl")
using .MyReverseDiff


In [3]:
#177.400 μs (9806 allocations: 497.84 KiB)
using BenchmarkTools

# @btime MyReverseDiff.multi_convolution!(gg,@view(xx[:,s:s]),mm)
#@btime MyReverseDiff.multi_convolution!(gg,xx,mm)
#@btime MyReverseDiff.multi_convolution_fast!(gg,xx,mm)
#@btime MyReverseDiff.multi_convolution_fourier(xx,mm)
# @btime MyReverseDiff.im2col(xx,m)
# @btime MyReverseDiff.im2col_p(xx,m)
#@btime MyReverseDiff.multi_convolution(xx,mm)

@btime MyReverseDiff.dif_convolution_fix(xx,mm,gg)

  11.295 ms (40255 allocations: 29.77 MiB)


(Float32[-3.5728562 0.29097152 … 0.74987197 1.1958877; -0.69869316 -0.10018628 … 1.7714828 -1.5283035; … ; 0.9397646 1.3882711 … -1.2553812 3.8321176; 1.7025809 1.8939843 … -2.2533724 1.8178376], Float32[119.9653 -140.69046 85.08795; -332.05716 79.25273 -42.02089; -123.08079 -82.07438 52.970436])

In [6]:
#Sprawdzić poprawność dif konwolucji
# Szerokosc - liczba obrazow, Wysokosc - dlugosc obrazow
a = Matrix([1.0f0 2.0f0 3.0f0 4.0f0]')
#a = Matrix([1.0f0 2.0f0 3.0f0 4.0f0]')
# Szerokosc - liczba masek, Wysokosc - dlugosc masek
m = Matrix([1.0f0 2.0f0; 1.0f0 0.0f0; 0.0f0 1.0f0]')

y = MyReverseDiff.multi_convolution(a,m)

g = ones(Float32,size(y))

d=MyReverseDiff.dif_convolution_fix(a,m,g)
dd=MyReverseDiff.dif_convolution(a,m,g)
println(y)
println(dd)
println(d)

Float32[5.0 1.0 2.0; 8.0 2.0 3.0; 11.0 3.0 4.0; 4.0 4.0 0.0]
(Float32[15.0; 15.0; 15.0; 6.0;;], Float32[4.0 4.0 4.0; 7.0 7.0 7.0])
(Float32[2.0; 5.0; 5.0; 5.0;;], Float32[10.0 10.0 10.0; 9.0 9.0 9.0])


Time:
- 812.372 ms (18244237 allocations: 2.30 GiB)
- 998.332 ms (18489737 allocations: 2.31 GiB)
- 726.103 ms (18489737 allocations: 2.31 GiB)
- 313.883 ms (6366137 allocations: 837.77 MiB)
- 325.212 ms (6366122 allocations: 826.32 MiB)
- 323.829 ms (6180722 allocations: 784.23 MiB)
- 343.182 ms (6000425 allocations: 776.92 MiB)
- 169.131 ms (427 allocations: 11.54 MiB)
- 14.702 ms (41638 allocations: 29.37 MiB)
- 13.583 ms (41438 allocations: 29.25 MiB)
- 10.206 ms (11000 allocations: 27.27 MiB)

In [1]:
include("../MyReverseDiff.jl")
include("../MyMlp.jl")

using .MyReverseDiff
using .MyMlp
using JLD2
using Printf
using Random

## Przygotowanie danych IMDB

In [2]:
X_train = Matrix(load("../../dataset/imdb_dataset_prepared.jld2", "X_train"));
y_train = Matrix(load("../../dataset/imdb_dataset_prepared.jld2", "y_train"));
X_test = Matrix(load("../../dataset/imdb_dataset_prepared.jld2", "X_test"));
y_test = Matrix(load("../../dataset/imdb_dataset_prepared.jld2", "y_test"));

input_size = size(X_train,1) # Liczba cech

17703

##  Trening modelu

In [3]:
batch_size = 64

model = Chain(
    ConvolutionBlock(3,3,3,name="layer1"),
    ConvolutionBlock(3,3,3,name="layer2"),
    FlattenBlock(name="flatten"),
    Dense(input_size*batch_size, batch_size, identity, name="softnet")
)

#   Utworzenie początkowych węzłów Constant dla danych wejściowych i etykiet
x_input_node = Constant(zeros(Float32, input_size, batch_size))
y_label_node = Constant(zeros(Float32, batch_size, 1))

#   Budowanie grafu treningowego
loss_node, model_output_node, order = build_graph!(model, binarycrossentropy, x_input_node, y_label_node; loss_name="loss")

optimizer_state = setup_optimizer(Adam(), model)


epochs = 5


for epoch in 1:epochs
    # --- Tasowanie zbioru treningowego NA NOWO w każdej epoce ---
    permutation = randperm(size(X_train, 2))
    X_train_shuffled_epoch = X_train[:, permutation]
    y_train_shuffled_epoch = y_train[:, permutation]
    num_batches = ceil(Int, size(X_train, 2) / batch_size)

    loss_value = 0.0

    t = @elapsed begin

    for i in 1:num_batches

        start_idx = (i - 1) * batch_size + 1
        end_idx = min(i * batch_size, size(X_train, 2))
        x_batch = X_train_shuffled_epoch[:, start_idx:end_idx]
        y_batch = y_train_shuffled_epoch[:, start_idx:end_idx]

        current_batch_size = size(x_batch, 2)
        view(x_input_node.output, :, 1:current_batch_size) .= x_batch
        view(y_label_node.output, 1:current_batch_size,:) .= y_batch'


        forward!(order)

        backward!(order)

        step!(optimizer_state)
        loss_value += loss_node.output

        println("Epoch: ", epoch, "Batches: ", i, "/",num_batches)
    end
end
    avg_loss_epoch = loss_value / num_batches

    println(@sprintf("Epoch: %d (%.2fs) \tTrain: (l: %.2f)", epoch, t, avg_loss_epoch))
end

OutOfMemoryError: OutOfMemoryError()